In [5]:
# Import some useful libraries
%matplotlib inline
import pandas as pd
import urllib
import xml.etree.ElementTree as ET
from scraper import *
import numpy as np
import scipy.sparse as sp
from tables import *
%load_ext autoreload
%autoreload 2

# display all pandas columns

pd.set_option('display.max_columns', 100)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Tables used for D3 visualisation 
Tables:
* author
* cosigner
* interest

In [8]:
df = pd.read_csv('data/Business.csv')

/Users/joachimmuth/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df.head()

,Unnamed: 0,BusinessShortNumber,BusinessStatus,BusinessStatusDate,BusinessStatusText,BusinessType,BusinessTypeAbbreviation,BusinessTypeName,Category,Description,DocumentationText,DraftText,FederalCouncilProposal,FederalCouncilProposalDate,FederalCouncilProposalText,FederalCouncilResponseText,FirstCouncil1,FirstCouncil1Abbreviation,FirstCouncil1Name,FirstCouncil2,FirstCouncil2Abbreviation,FirstCouncil2Name,ID,InitialSituation,IsLeadingDepartment,Language,Modified,MotionText,Proceedings,ReasonText,ResponsibleDepartment,ResponsibleDepartmentAbbreviation,ResponsibleDepartmentName,SubmissionCouncil,SubmissionCouncilAbbreviation,SubmissionCouncilName,SubmissionDate,SubmissionLegislativePeriod,SubmissionSession,SubmittedBy,SubmittedText,TagNames,Tags,Title
0,0,11.758,27,2000-09-26T00:00:00,Liquidé,3,Iv.ct.,Initiative cantonale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CN,Conseil national,NaN,NaN,NaN,19110758,NaN,True,FR,2010-12-26T13:31:48.703,NaN,NaN,NaN,4.0,DFI,Département de l'intérieur,NaN,NaN,NaN,1973-08-15T00:00:00,39.0,4311,Berne,<text><p>La législation fédérale doit être dév...,NaN,NaN,Médicaments. Législation
1,1,78.222,27,1981-03-10T00:00:00,Liquidé,4,Iv.pa.,Initiative parlementaire,NaN,Rapport de la Commission du 27.08.1979,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CN,Conseil national,NaN,NaN,NaN,19780222,NaN,NaN,FR,2010-12-26T13:31:48.703,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CN,Conseil national,1978-06-05T00:00:00,40.0,4311,NaN,NaN,NaN,NaN,Code pénal. Interruption de la grossesse (Gira...
2,2,80.226,27,1983-03-03T00:00:00,Liquidé,4,Iv.pa.,Initiative parlementaire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CN,Conseil national,NaN,NaN,NaN,19800226,NaN,True,FR,2010-12-26T13:31:48.703,NaN,NaN,NaN,1.0,Parl,Parlement,1.0,CN,Conseil national,1980-06-02T00:00:00,41.0,4311,NaN,NaN,NaN,NaN,Constitutiion fédérale. Droit de l'entreprise ...
3,3,85.019,27,2003-12-16T00:00:00,Liquidé,1,OCF,Objet du Conseil fédéral,V,Message et projet d'arrêté du 1er mai 1985 con...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CN,Conseil national,NaN,NaN,NaN,19850019,<text><p>L'accord établit le cadre de droit in...,True,FR,2011-10-19T17:15:58.637,NaN,<text><p></p><p>Les deux conseils ont décidé s...,NaN,3.0,DFAE,Département des affaires étrangères,NaN,NaN,NaN,1985-05-01T00:00:00,42.0,4311,NaN,NaN,NaN,NaN,Utilisation pacifique de l'énergie nucléaire. ...
4,4,85.227,27,2000-10-06T00:00:00,Liquidé,4,Iv.pa.,Initiative parlementaire,III,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,CE,Conseil des Etats,NaN,NaN,NaN,19850227,<text><p>Le droit suisse des assurances social...,True,FR,2011-10-19T17:17:52.9,NaN,<text><p>Le <b>Conseil des Etats</b> a accepté...,NaN,1.0,Parl,Parlement,2.0,CE,Conseil des Etats,1985-02-07T00:00:00,42.0,4311,Meier Josi J.,<text><p>A la suite de la motion visant une me...,NaN,NaN,Droit des assurances sociales


In [6]:
tables = Tables()

/Users/joachimmuth/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (10,27) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
interest = tables.interest()
interest.head()

KeyError: 2500

In [4]:
authors = tables.author()
authors.head()

,MemberCouncilNumber,author
0,1,42
1,2,6
2,6,1
3,7,10
4,8,5


In [7]:
cosigners = tables.cosigner()
cosigners

Cosigners table shape:  (316548, 2)
Authors table shape:  (31095, 2)
Matrix created of size  (array([   1,    1,    1, ..., 4209, 4209, 4209], dtype=int32), array([   1,    8,    9, ..., 4200, 4202, 4208], dtype=int32))


<4210x4210 sparse matrix of type '<class 'numpy.int32'>'
	with 100529 stored elements in LInked List format>

## Explanation of the Table

* **Business** is the table of initiatives
* **BusinessRole** table of all the person who act for or against a business. IMPORTANT! **BusinessRole.MemberCouncilNumber** link to **MemberCouncil.PersonNumber**. **BusinessShortNumber** link businesses and roles.
    * Auteur
    * Cosignataire
    * Porte-parole
    * Opposant(e)
    * Repreneur
    
* **BusinessType** contain all 18 types of businesses. As the type is reapeated in **Business.BusinessType** and **Business.BusinessTypeName**, it is not useful for us.
* **BusinessStatus** give the current status of a business. Again, this information is given in **Business.BusinessStatusText**, so not useful


* Tag are stored in "TagNames" Array
* **Tags** are easier (as it is integer) 

# Compiling the _Cosignataire_ info into an adjacency matrix for d3

In [ ]:
adj = pd.read_csv('data/adj.csv')

In [ ]:
adj.rename(columns={'PersonNumber': 'PersonIdCode'}, inplace=True)

In [ ]:
adj.set_index('PersonIdCode', inplace=True)

In [ ]:
adj.head(2)

In [ ]:
people = pd.read_csv('data/people_jonas.csv').set_index('PersonIdCode')

In [ ]:
people.loc[2023].PersonNumber

In [ ]:
def fill_adj(adj, people):
    # getting a list of active members (we're only interested in them)
    active = people.PersonNumber.tolist()
    # going through an empty adj matrix with PersonIdCodes as rows and columns
    for row in adj.iterrows():
        person_id = row[0]
        # converting from PersonIdCode to PersonNumber for friends search
        person_number = people.loc[person_id].PersonNumber
        # searching co-sign friends w/ the function defined above
        friends_matrix = get_friends(friends, person_number)
        # looping through friends to fill the matrix
        for friend in friends_matrix.iterrows():
            # checking if active
            if friend[1].Person_num in active:
                # converting from PersonNumber to PersonIdCode
                friend_id = people.loc[people.PersonNumber == friend[1].Person_num].index.tolist()[0]
                # Updating matrix
                adj.loc[person_id, str(friend_id)] = friend[1].times_cosigner

In [ ]:
fill_adj(adj, people)

## WARNING: the adjacency matrix is **NOT** symmetric

For now, I'll be only considering rows in the viz, but might make sense to consider columns, dunno.

In [ ]:
adj.head(2)

In [ ]:
adj.to_csv('data/adj_cosign.csv')

In [ ]:
adj.to_json('data/adj_cosign.json', orient='index')